### Analysis



In [28]:
# mongoimport --type csv -d nystateinpatientdischarges2022 -c NY_data_2022 --headerline --drop NY_data_2022.csv

In [29]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pymongo import MongoClient
from pprint import pprint

In [30]:
# Extrating Raw Data for NY State for 2020 
csv_file_path_2020 = '../NY-Healthcare-Data-Visualization/Resources/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2020.csv'
df_2020 = pd.read_csv(csv_file_path_2020)
df_2020.head()

C:\Users\rtkam\AppData\Local\Temp\ipykernel_40896\1608559429.py:3: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv(csv_file_path_2020)


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,50 to 69,104,F,Black/African American,Not Span/Hispanic,...,Moderate,Moderate,Medical,Medicare,NaN,NaN,NaN,Y,214778.30,40532.42
1,New York City,Bronx,7000006.0,1168.0,Montefiore Medical Center-Wakefield Hospital,18 to 29,104,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,N,33385.21,8996.74
2,New York City,Bronx,7000006.0,1168.0,Montefiore Medical Center-Wakefield Hospital,50 to 69,104,F,Other Race,Spanish/Hispanic,...,Moderate,Major,Medical,Medicare,Medicaid,NaN,NaN,Y,16581.52,1735.53
3,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,0 to 17,104,M,Other Race,Spanish/Hispanic,...,Moderate,Moderate,Medical,Medicaid,NaN,NaN,NaN,N,14086.86,3438.63
4,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,0 to 17,104,M,Other Race,Spanish/Hispanic,...,Moderate,Minor,Medical,Blue Cross/Blue Shield,NaN,NaN,NaN,Y,28222.99,6886.04


In [31]:
#Extrating the latitude and longitude for the NY facilities
NY_lat_long='../NY-Healthcare-Data-Visualization/Resources/NY_Health_Facility_Lat_Long.csv'
NY_lat_long_df=pd.read_csv(NY_lat_long)
NY_lat_long_df.head(10)

,Facility Name,Latitude,Longitude
0,Montefiore Med Center - Jack D Weiler Hosp of ...,40.849278,-73.846121
1,Montefiore Medical Center-Wakefield Hospital,40.894103,-73.861005
2,Montefiore Medical Center - Henry & Lucy Moses...,40.880839,-73.878453
3,Montefiore Nyack,41.096191,-73.925989
4,New York Presbyterian Hospital - Allen Hospital,40.873581,-73.913139
5,Mount Sinai Beth Israel,40.733875,-73.982099
6,Mount Sinai West,40.772391,-73.986219
7,White Plains Hospital Center,41.026875,-73.769145
8,New York-Presbyterian Hospital - Columbia Pres...,40.842039,-73.941232
9,Kings County Hospital Center,40.657492,-73.942921


In [32]:
# Extrating Raw Data for NY State for 2021
csv_file_path_2021 = '../NY-Healthcare-Data-Visualization/Resources/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2021.csv'
df_2021 = pd.read_csv(csv_file_path_2021)
df_2021.head()

C:\Users\rtkam\AppData\Local\Temp\ipykernel_40896\864287361.py:3: DtypeWarning: Columns (10,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv(csv_file_path_2021)


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,70 or Older,104,M,Other Race,Spanish/Hispanic,...,Major,Extreme,Medical,Medicare,Medicaid,NaN,NaN,Y,320922.43,60241.34
1,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,50 to 69,104,F,White,Not Span/Hispanic,...,Moderate,Minor,Medical,Private Health Insurance,NaN,NaN,NaN,Y,61665.22,9180.69
2,New York City,Bronx,7000006.0,1168.0,Montefiore Medical Center-Wakefield Hospital,18 to 29,104,F,Other Race,Spanish/Hispanic,...,Minor,Minor,Surgical,Medicaid,NaN,NaN,NaN,N,42705.34,11366.50
3,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,70 or Older,104,M,Other Race,Spanish/Hispanic,...,Major,Major,Medical,Medicare,Medicaid,NaN,NaN,Y,72700.17,12111.75
4,New York City,Bronx,7000006.0,1169.0,Montefiore Medical Center - Henry & Lucy Moses...,50 to 69,104,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicare,Medicaid,NaN,NaN,Y,55562.51,8339.72


In [33]:
# Extrating Raw Data for NY State for 2022 
csv_file_path_2022 = '../NY-Healthcare-Data-Visualization/Resources/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2022.csv'
df_2022 = pd.read_csv(csv_file_path_2022)
df_2022.head()

C:\Users\rtkam\AppData\Local\Temp\ipykernel_40896\4061537568.py:3: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv(csv_file_path_2022)


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,50 to 69,107,F,White,Not Span/Hispanic,...,Major,Major,Medical,Medicaid,NaN,NaN,NaN,Y,"51,514.62","7,552.54"
1,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,104,M,Black/African American,Spanish/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,Y,"25,370.86","3,469.55"
2,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,104,F,Other Race,Spanish/Hispanic,...,Minor,Minor,Medical,Medicaid,NaN,NaN,NaN,N,"23,876.78","6,180.33"
3,New York City,Bronx,7000006.0,3058.0,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,100,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,Y,"43,319.05","12,588.93"
4,New York City,Bronx,7000006.0,1168.0,Montefiore Medical Center-Wakefield Hospital,18 to 29,104,M,Other Race,Spanish/Hispanic,...,Moderate,Moderate,Medical,Medicaid,NaN,NaN,NaN,Y,"40,266.23","10,355.99"


In [34]:
selected_columns = ['Hospital Service Area', 'Hospital County', 'Facility Name', 'Age Group', 'Gender', 'Race',
       'Ethnicity', 'Length of Stay', 'APR MDC Description', 'APR Severity of Illness Code', 'Payment Typology 1',
       'Payment Typology 2', 'Payment Typology 3', 'Emergency Department Indicator', 'Total Charges', 'Total Costs']
clean_2022_df = df_2022[selected_columns]
clean_2022_df.head()

,Hospital Service Area,Hospital County,Facility Name,Age Group,Gender,Race,Ethnicity,Length of Stay,APR MDC Description,APR Severity of Illness Code,Payment Typology 1,Payment Typology 2,Payment Typology 3,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,50 to 69,F,White,Not Span/Hispanic,3,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,3,Medicaid,NaN,NaN,Y,"51,514.62","7,552.54"
1,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,M,Black/African American,Spanish/Hispanic,1,"DISEASES AND DISORDERS OF THE SKIN, SUBCUTANEO...",2,Medicaid,NaN,NaN,Y,"25,370.86","3,469.55"
2,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,F,Other Race,Spanish/Hispanic,2,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",1,Medicaid,NaN,NaN,N,"23,876.78","6,180.33"
3,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,F,Black/African American,Not Span/Hispanic,2,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",2,Medicaid,NaN,NaN,Y,"43,319.05","12,588.93"
4,New York City,Bronx,Montefiore Medical Center-Wakefield Hospital,18 to 29,M,Other Race,Spanish/Hispanic,4,MENTAL DISEASES AND DISORDERS,2,Medicaid,NaN,NaN,Y,"40,266.23","10,355.99"


In [35]:
need_to_drop = clean_2022_df[clean_2022_df['Facility Name'] == 'Redacted for Confidentiality']
need_to_drop.head()

,Hospital Service Area,Hospital County,Facility Name,Age Group,Gender,Race,Ethnicity,Length of Stay,APR MDC Description,APR Severity of Illness Code,Payment Typology 1,Payment Typology 2,Payment Typology 3,Emergency Department Indicator,Total Charges,Total Costs
142,NaN,NaN,Redacted for Confidentiality,18 to 29,F,Other Race,Not Span/Hispanic,1,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",2,Private Health Insurance,NaN,NaN,Y,"67,132.55","11,622.28"
991,NaN,NaN,Redacted for Confidentiality,0 to 17,F,Black/African American,Not Span/Hispanic,3,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",2,Medicaid,NaN,NaN,N,"24,899.29","11,378.45"
1171,NaN,NaN,Redacted for Confidentiality,18 to 29,F,Other Race,Unknown,4,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",3,Medicaid,NaN,NaN,Y,"94,310.37","22,860.04"
3258,NaN,NaN,Redacted for Confidentiality,18 to 29,F,Other Race,Spanish/Hispanic,1,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",2,Medicaid,NaN,NaN,Y,"30,437.00","6,921.89"
3944,NaN,NaN,Redacted for Confidentiality,18 to 29,M,Multi-racial,Not Span/Hispanic,1,DISEASES AND DISORDERS OF THE NERVOUS SYSTEM,2,Medicaid,NaN,NaN,Y,"13,791.09","8,814.23"


In [36]:
clean_data = clean_2022_df[~clean_2022_df["Facility Name"].isin(need_to_drop["Facility Name"])]
clean_data.head()

#clean_data.to_csv("Resources/NY_data_2022.csv", index=False)

,Hospital Service Area,Hospital County,Facility Name,Age Group,Gender,Race,Ethnicity,Length of Stay,APR MDC Description,APR Severity of Illness Code,Payment Typology 1,Payment Typology 2,Payment Typology 3,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,50 to 69,F,White,Not Span/Hispanic,3,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,3,Medicaid,NaN,NaN,Y,"51,514.62","7,552.54"
1,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,M,Black/African American,Spanish/Hispanic,1,"DISEASES AND DISORDERS OF THE SKIN, SUBCUTANEO...",2,Medicaid,NaN,NaN,Y,"25,370.86","3,469.55"
2,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,F,Other Race,Spanish/Hispanic,2,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",1,Medicaid,NaN,NaN,N,"23,876.78","6,180.33"
3,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,F,Black/African American,Not Span/Hispanic,2,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",2,Medicaid,NaN,NaN,Y,"43,319.05","12,588.93"
4,New York City,Bronx,Montefiore Medical Center-Wakefield Hospital,18 to 29,M,Other Race,Spanish/Hispanic,4,MENTAL DISEASES AND DISORDERS,2,Medicaid,NaN,NaN,Y,"40,266.23","10,355.99"


In [38]:
clean_data_merged=clean_data.merge(NY_lat_long_df, on='Facility Name', how="left")
clean_data_merged.head()

,Hospital Service Area,Hospital County,Facility Name,Age Group,Gender,Race,Ethnicity,Length of Stay,APR MDC Description,APR Severity of Illness Code,Payment Typology 1,Payment Typology 2,Payment Typology 3,Emergency Department Indicator,Total Charges,Total Costs,Latitude,Longitude
0,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,50 to 69,F,White,Not Span/Hispanic,3,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,3,Medicaid,NaN,NaN,Y,"51,514.62","7,552.54",40.849278,-73.846121
1,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,M,Black/African American,Spanish/Hispanic,1,"DISEASES AND DISORDERS OF THE SKIN, SUBCUTANEO...",2,Medicaid,NaN,NaN,Y,"25,370.86","3,469.55",40.849278,-73.846121
2,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,F,Other Race,Spanish/Hispanic,2,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",1,Medicaid,NaN,NaN,N,"23,876.78","6,180.33",40.849278,-73.846121
3,New York City,Bronx,Montefiore Med Center - Jack D Weiler Hosp of ...,18 to 29,F,Black/African American,Not Span/Hispanic,2,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",2,Medicaid,NaN,NaN,Y,"43,319.05","12,588.93",40.849278,-73.846121
4,New York City,Bronx,Montefiore Medical Center-Wakefield Hospital,18 to 29,M,Other Race,Spanish/Hispanic,4,MENTAL DISEASES AND DISORDERS,2,Medicaid,NaN,NaN,Y,"40,266.23","10,355.99",40.894103,-73.861005


In [39]:
column_name = "Facility Name"
unique_values = clean_data[column_name].unique()
print("Unique values in", column_name, ":", unique_values)

Unique values in Facility Name : ['Montefiore Med Center - Jack D Weiler Hosp of A Einstein College Div'
 'Montefiore Medical Center-Wakefield Hospital'
 'Montefiore Medical Center - Henry & Lucy Moses Div' 'Montefiore Nyack'
 'New York Presbyterian Hospital - Allen Hospital'
 'Mount Sinai Beth Israel' 'Mount Sinai West'
 'White Plains Hospital Center'
 'New York-Presbyterian Hospital - Columbia Presbyterian Center'
 'Kings County Hospital Center' 'Bellevue Hospital Center'
 'North Central Bronx Hospital' 'Mount Sinai Hospital'
 'Jacobi Medical Center' 'Coney Island Hospital' 'Mount Sinai Brooklyn'
 'Henry J. Carter Specialty Hospital'
 'Lincoln Medical & Mental Health Center' 'Metropolitan Hospital Center'
 'Queens Hospital Center' 'Elmhurst Hospital Center'
 'Harlem Hospital Center' 'Woodhull Medical & Mental Health Center'
 'Montefiore Mount Vernon Hospital' 'Montefiore New Rochelle Hospital'
 'New York-Presbyterian Westchester Behavioral Health Center'
 'New York-Presbyterian/Lower

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that the "autosaurus" database is in MongoDB
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'nystateinpatientdischarges2022', 'petsitly_marketing', 'uk_food']


In [ ]:
# assign the database to a variable name
db = mongo['nystateinpatientdischarges2022']

In [ ]:
# review the collections in our new database
collection = print(db.list_collection_names())
collection

['NY_data_2022']


In [ ]:
# review a document in the customer_list collection
print(db.NY_data_2022.find_one())

{'_id': ObjectId('65d65e2b15dfe7ded99405f0'), 'Hospital Service Area': 'New York City', 'Hospital County': 'Bronx', 'Facility Name': 'Montefiore Med Center - Jack D Weiler Hosp of A Einstein College Div', 'Age Group': '50 to 69', 'Gender': 'F', 'Race': 'White', 'Ethnicity': 'Not Span/Hispanic', 'Length of Stay': 3, 'APR MDC Description': 'DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM', 'APR Severity of Illness Code': 3, 'Payment Typology 1': 'Medicaid', 'Payment Typology 2': '', 'Payment Typology 3': '', 'Emergency Department Indicator': 'Y', 'Total Charges': '51,514.62', 'Total Costs': '7,552.54'}
